In [5]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

if torch.cuda.is_available():
    print(f"GPU device name: {torch.cuda.get_device_name(0)}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")

Using cuda device
GPU device name: NVIDIA GeForce RTX 4050 Laptop GPU
Number of GPUs: 1


In [ ]:
# Define the class
# https://docs.pytorch.org/docs/stable/nn.html
class NeuralNetwork(nn.Module): # inherit from nn.Module
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [9]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [10]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0040,  0.0197, -0.0004,  ...,  0.0274, -0.0298,  0.0313],
        [-0.0350,  0.0287, -0.0110,  ..., -0.0283,  0.0141, -0.0282]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0339, -0.0002], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0036,  0.0161,  0.0226,  ...,  0.0077, -0.0098,  0.0204],
        [ 0.0038, -0.0167, -0.0260,  ...,  0.0106,  0.0043,  0.0081]],
       device='cuda:0', grad_fn=<Sl

In [16]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
print(logits.shape)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

torch.Size([1, 10])
Predicted class: tensor([2], device='cuda:0')


In [ ]:
input_image = torch.rand(3,28,28) # batch_size * img_width * img_height
print(input_image.size())

torch.Size([3, 28, 28])


In [18]:
# nn.Flatten()
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [ ]:
# nn.Linear()
layer1 = nn.Linear(in_features=28*28, out_features=20) # @ 784*20
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [20]:
# nn.ReLU
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.0940, -0.1705,  0.0812, -0.1921, -0.6103, -0.1234,  0.7229,  0.1072,
          0.4953, -0.2916, -0.5115,  0.1883, -0.1415, -0.2691,  0.0160,  0.1573,
         -0.4164,  0.3414,  0.3294, -0.1915],
        [ 0.4049,  0.1545,  0.1057, -0.2869, -0.5996,  0.1379,  0.7120, -0.1825,
          0.0676, -0.1951, -0.3428, -0.1930, -0.2864, -0.0875, -0.1428,  0.3625,
         -0.3732,  0.0176, -0.0642,  0.1467],
        [ 0.4307, -0.2329,  0.4490,  0.0308, -0.4703,  0.2883,  0.6345, -0.1573,
          0.4207,  0.2304, -0.4535,  0.1989, -0.1487, -0.0272, -0.4206,  0.3876,
         -0.1385,  0.2882, -0.0918,  0.3133]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0940, 0.0000, 0.0812, 0.0000, 0.0000, 0.0000, 0.7229, 0.1072, 0.4953,
         0.0000, 0.0000, 0.1883, 0.0000, 0.0000, 0.0160, 0.1573, 0.0000, 0.3414,
         0.3294, 0.0000],
        [0.4049, 0.1545, 0.1057, 0.0000, 0.0000, 0.1379, 0.7120, 0.0000, 0.0676,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.00

In [23]:
# nn.Sequential
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
